In [2]:
from datetime import timedelta
import pandas as pd

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
import numpy as np
from scipy.stats import linregress
from tqdm import tqdm

TOKEN = "token"

In [3]:
with Client(TOKEN) as client:
    shares = client.instruments.shares().instruments
    shares = [i for i in shares if i.currency == "rub"]

In [4]:
TARGET_COLUMNS = ["figi", "ticker", "lot", "name", "sector"]
shares_df = pd.DataFrame(columns=TARGET_COLUMNS)
for i, share in enumerate(shares):
    shares_df.loc[i] = pd.Series(
        (share.figi, share.ticker, share.lot, share.name, share.sector), TARGET_COLUMNS
    )
shares_df

,figi,ticker,lot,name,sector
0,TCS007940839,KZIZP,1,Красногорский завод им. С.А. Зверева - привиле...,other
1,TCS00A108ZR8,DATA,1,Группа Аренадата,it
2,BBG000FWGSZ5,IRKT,100,Яковлев,industrials
3,BBG004S68CV8,VSMO,1,ВСМПО-АВИСМА,materials
4,BBG000Q7ZZY2,UNAC,1000,Объединенная авиастроительная корпорация,industrials
...,...,...,...,...,...
167,BBG004S68BR5,NMTP,100,НМТП,industrials
168,BBG000QFH687,TGKA,100000,ТГК-1,utilities
169,BBG0027F0Y27,CNTLP,100,Центральный Телеграф - акции привилегированные,telecom
170,BBG004S68FR6,MTLRP,10,Мечел - Привилегированные акции,materials


In [5]:
portfolio_df = pd.DataFrame()
with Client(TOKEN) as client:
    candles_ticker = {}
    for index, row in tqdm(shares_df.iterrows()):
        figi, ticker, lot, name, sector = row
        candles = map(
            lambda x: (x.time, x.close.units + x.close.nano / 1e9),
            client.get_all_candles(
                figi=figi,
                from_=now() - timedelta(days=30),
                interval=CandleInterval.CANDLE_INTERVAL_DAY,
            ),
        )
        candles_data = []
        for _, close in candles:
            candles_data.append(close)
        candles_ticker[str(ticker)] = candles_data

172it [00:07, 22.16it/s]


In [6]:
for i in candles_ticker:
    candles_data = candles_ticker[i]
    if len(candles_data) == 0:
        continue
    x = np.arange(len(candles_data))
    y = np.array(candles_data)

    slope, intercept, r_value, p_value, std_err = linregress(x, y)

    if slope > 0:
        shares_df.loc[shares_df["ticker"] == i, "rating"] = slope
    else:
        shares_df.loc[shares_df["ticker"] == i, "rating"] = 0
shares_df = shares_df[shares_df["rating"] != 0]
shares_df.head()

,figi,ticker,lot,name,sector,rating
1,TCS00A108ZR8,DATA,1,Группа Аренадата,it,0.948131
18,TCS2207L1061,HHRU,1,HeadHunter Group PLC,it,NaN
28,BBG008F2T3T2,RUAL,10,РУСАЛ,materials,0.051245
59,BBG002YFXL29,UNKL,1,ЮУНК,materials,24.381705
68,TCS009084453,NOMP,1,Новошип - ао,industrials,0.298013


In [7]:
with Client(TOKEN) as client:
    for _, row in tqdm(shares_df.iterrows()):
        price = client.market_data.get_last_prices(figi=[row.figi]).last_prices[0]
        price = price.price.units + price.price.nano / 10e9
        price *= row.lot
        if price == 0:
            continue
        shares_df.loc[shares_df["ticker"] == row["ticker"], "price"] = price
shares_df.sort_values(by="rating", ascending=False, inplace=True)
shares_df.head()

0it [00:00, ?it/s]/var/folders/5w/jv8qq0fn5b3dwhxy_06sbt7m0000gn/T/ipykernel_43759/1872939268.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shares_df.loc[shares_df["ticker"] == row["ticker"], "price"] = price
18it [00:00, 32.44it/s]
/var/folders/5w/jv8qq0fn5b3dwhxy_06sbt7m0000gn/T/ipykernel_43759/1872939268.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shares_df.sort_values(by="rating", ascending=False, inplace=True)


,figi,ticker,lot,name,sector,rating,price
161,BBG000R607Y3,PLZL,1,Полюс,materials,110.324675,15259.050
121,BBG004S688G4,AKRN,1,Акрон,materials,39.296443,17012.000
59,BBG002YFXL29,UNKL,1,ЮУНК,materials,24.381705,6110.000
106,TCS20A107662,HEAD,1,Хэдхантер,it,4.446076,4261.000
1,TCS00A108ZR8,DATA,1,Группа Аренадата,it,0.948131,123.002


In [13]:
balance = 1_000_000  
portfolio_balance = 0
portfolio = {}

with Client(TOKEN) as client:
    for _, row in tqdm(shares_df.iterrows(), total=shares_df.shape[0]):
        if portfolio_balance + row.price <= balance:
            portfolio[f"{row.ticker}"] = portfolio.get(row.ticker, 0) + 1
            portfolio_balance += row.price

    for _, row in tqdm(shares_df.iterrows(), total=shares_df.shape[0]):
        j = 1
        while portfolio_balance + row.price * j <= balance:
            portfolio[row.ticker] = portfolio.get(row.ticker, 0) + 1
            portfolio_balance += row.price
            j += 1

print(portfolio)
print("Balance: ", portfolio_balance)

100%|██████████| 18/18 [00:00<00:00, 186.82it/s]

{'PLZL': 32, 'AKRN': 15, 'UNKL': 20, 'HEAD': 15, 'DATA': 254, 'GLTR': 31, 'LSRG': 12, 'NOMP': 18, 'QIWI': 10, 'RUAL': 4, 'VEON-RX': 3, 'AFLT': 1, 'SFIN': 1, 'SGZH': 2, 'MRKC': 3, 'HHRU': 1, 'GRNT': 7800}
Balance:  999939.9469996699
